# Exploring Ensemble Methods

In this assignment, we will explore the use of boosting. We will use the pre-implemented gradient boosted trees in GraphLab Create. You will:

* Use SFrames to do some feature engineering.
* Train a boosted ensemble of decision-trees (gradient boosted trees) on the LendingClub dataset.
* Predict whether a loan will default along with prediction probabilities (on a validation set).
* Evaluate the trained model and compare it with a baseline.
* Find the most positive and negative loans using the learned model.
* Explore how the number of trees influences classification performance.

Let's get started!

## Fire up Graphlab Create

In [53]:
import graphlab
import numpy
import pandas
import seaborn
from tabulate import tabulate

In [58]:
%matplotlib inline
seaborn.set_style('whitegrid')

# Load LendingClub dataset

We will be using the [LendingClub](https://www.lendingclub.com/) data. As discussed earlier, the [LendingClub](https://www.lendingclub.com/) is a peer-to-peer leading company that directly connects borrowers and potential lenders/investors. 

Just like we did in previous assignments, we will build a classification model to predict whether or not a loan provided by lending club is likely to default.

Let us start by loading the data.

In [18]:
loans = graphlab.SFrame('../common/lending-club-data.gl/')
loan_frame = pandas.read_csv('../common/lending-club-data.csv')

/home/cronos/.virtualenvs/machinelearning/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2723: DtypeWarning: Columns (19,47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [26]:
for column_index in (19, 47):
    column = loan_frame.columns[column_index]
    print(column)
    print('-' * len(column))
    print(loan_frame[column].describe())
    print('')

desc
----
count     60706
unique    60315
top            
freq        227
Name: desc, dtype: object

next_pymnt_d
------------
count                2907
unique                 89
top       20150301T000000
freq                  131
Name: next_pymnt_d, dtype: object



In [36]:
loan_frame.shape

(122607, 68)

Let's quickly explore what the dataset looks like. First, let's print out the column names to see what features we have in this dataset. We have done this in previous assignments, so we won't belabor this here.

In [20]:
for column in loan_frame.columns:
    print("{0}\t{1}".format(column, loan_frame[column].dtype))

id	int64
member_id	int64
loan_amnt	int64
funded_amnt	int64
funded_amnt_inv	int64
term	object
int_rate	float64
installment	float64
grade	object
sub_grade	object
emp_title	object
emp_length	object
home_ownership	object
annual_inc	float64
is_inc_v	object
issue_d	object
loan_status	object
pymnt_plan	object
url	object
desc	object
purpose	object
title	object
zip_code	object
addr_state	object
dti	float64
delinq_2yrs	float64
earliest_cr_line	object
inq_last_6mths	float64
mths_since_last_delinq	float64
mths_since_last_record	float64
open_acc	float64
pub_rec	float64
revol_bal	int64
revol_util	float64
total_acc	float64
initial_list_status	object
out_prncp	float64
out_prncp_inv	float64
total_pymnt	float64
total_pymnt_inv	float64
total_rec_prncp	float64
total_rec_int	float64
total_rec_late_fee	float64
recoveries	float64
collection_recovery_fee	float64
last_pymnt_d	object
last_pymnt_amnt	float64
next_pymnt_d	object
last_credit_pull_d	object
collections_12_mths_ex_med	float64
mths_since_last_major_de

## Modifying the target column

The target column (label column) of the dataset that we are interested in is called `bad_loans`. In this column **1** means a risky (bad) loan **0** means a safe  loan.

As in past assignments, in order to make this more intuitive and consistent with the lectures, we reassign the target to be:
* **+1** as a safe  loan, 
* **-1** as a risky (bad) loan. 

We put this in a new column called `safe_loans`.

In [42]:
if 'bad_loans' in loans.column_names():
    loans['safe_loans'] = loans['bad_loans'].apply(lambda x : +1 if x==0 else -1)
    loans = loans.remove_column('bad_loans')

loan_frame['safe_loans'] = loan_frame.bad_loans.apply(lambda x: 1 if x==0 else -1)

## Selecting features

In this assignment, we will be using a subset of features (categorical and numeric). The features we will be using are **described in the code comments** below. If you are a finance geek, the [LendingClub](https://www.lendingclub.com/) website has a lot more details about these features.

The features we will be using are described in the code comments below:

In [7]:
target = 'safe_loans'
features = ['grade',                     # grade of the loan (categorical)
            'sub_grade_num',             # sub-grade of the loan as a number from 0 to 1
            'short_emp',                 # one year or less of employment
            'emp_length_num',            # number of years of employment
            'home_ownership',            # home_ownership status: own, mortgage or rent
            'dti',                       # debt to income ratio
            'purpose',                   # the purpose of the loan
            'payment_inc_ratio',         # ratio of the monthly payment to income
            'delinq_2yrs',               # number of delinquincies 
            'delinq_2yrs_zero',          # no delinquincies in last 2 years
            'inq_last_6mths',            # number of creditor inquiries in last 6 months
            'last_delinq_none',          # has borrower had a delinquincy
            'last_major_derog_none',     # has borrower had 90 day or worse rating
            'open_acc',                  # number of open credit accounts
            'pub_rec',                   # number of derogatory public records
            'pub_rec_zero',              # no derogatory public records
            'revol_util',                # percent of available credit being used
            'total_rec_late_fee',        # total late fees received to day
            'int_rate',                  # interest rate of the loan
            'total_rec_int',             # interest received to date
            'annual_inc',                # annual income of borrower
            'funded_amnt',               # amount committed to the loan
            'funded_amnt_inv',           # amount committed by investors for the loan
            'installment',               # monthly payment owed by the borrower
           ]

## Skipping observations with missing values

In [33]:
for column in features:
    if loan_frame[column].hasnans:
        print(column)
        print('-' * len(column))
        cleaned = loan_frame[column].dropna()
        print("Nans: {0}".format(len(loan_frame[column]) - len(cleaned)))
        print('')

payment_inc_ratio
-----------------
Nans: 4

delinq_2yrs
-----------
Nans: 29

delinq_2yrs_zero
----------------
Nans: 29

inq_last_6mths
--------------
Nans: 29

open_acc
--------
Nans: 29

pub_rec
-------
Nans: 29

pub_rec_zero
------------
Nans: 29

annual_inc
----------
Nans: 4



Recall from the lectures that one common approach to coping with missing values is to **skip** observations that contain missing values.

We run the following code to do so:

In [8]:
loans, loans_with_na = loans[[target] + features].dropna_split()

# Count the number of rows with missing data
num_rows_with_na = loans_with_na.num_rows()
num_rows = loans.num_rows()
print 'Dropping %s observations; keeping %s ' % (num_rows_with_na, num_rows)

Dropping 29 observations; keeping 122578 


Fortunately, there are not too many missing values. We are retaining most of the data.

## Make sure the classes are balanced

In [73]:
loan_frame['safe_unsafe'] = loan_frame.bad_loans.apply(lambda x: 'safe' if x == 0 else 'risky')
safe_unsafe_proportions = loan_frame.safe_unsafe.value_counts(normalize=True)
safe_unsafe_count = loan_frame.safe_unsafe.value_counts()
print(tabulate({"safe or unsafe":safe_unsafe_proportions.index,
                "proportion": safe_unsafe_proportions.values},
               headers='keys',
               tablefmt='psql'))
print(tabulate({"safe or unsafe": safe_unsafe_count.index,
                "count":safe_unsafe_count.values},
               headers='keys',
               tablefmt='psql'))

+------------------+--------------+
| safe or unsafe   |   proportion |
|------------------+--------------|
| safe             |     0.811185 |
| risky            |     0.188815 |
+------------------+--------------+
+---------+------------------+
|   count | safe or unsafe   |
|---------+------------------|
|   99457 | safe             |
|   23150 | risky            |
+---------+------------------+


In [65]:
axe = safe_unsafe_counts.plot(kind='bar', title='Proportion of Loans Safe or Risky')

We saw in an earlier assignment that this dataset is also imbalanced. We will undersample the larger class (safe loans) in order to balance out our dataset. We used `seed=1` to make sure everyone gets the same results.

In [9]:
safe_loans_raw = loans[loans[target] == 1]
risky_loans_raw = loans[loans[target] == -1]

# Undersample the safe loans.
percentage = len(risky_loans_raw)/float(len(safe_loans_raw))
safe_loans = safe_loans_raw.sample(percentage, seed = 1)
risky_loans = risky_loans_raw
loans_data = risky_loans.append(safe_loans)

print "Percentage of safe loans                 :", len(safe_loans) / float(len(loans_data))
print "Percentage of risky loans                :", len(risky_loans) / float(len(loans_data))
print "Total number of loans in our new dataset :", len(loans_data)

Percentage of safe loans                 : 0.502247166849
Percentage of risky loans                : 0.497752833151
Total number of loans in our new dataset : 46503


**Checkpoint:** You should now see that the dataset is balanced (approximately 50-50 safe vs risky loans).

**Note:** There are many approaches for dealing with imbalanced data, including some where we modify the learning algorithm. These approaches are beyond the scope of this course, but some of them are reviewed in this [paper](http://ieeexplore.ieee.org/xpl/login.jsp?tp=&arnumber=5128907&url=http%3A%2F%2Fieeexplore.ieee.org%2Fiel5%2F69%2F5173046%2F05128907.pdf%3Farnumber%3D5128907 ). For this assignment, we use the simplest possible approach, where we subsample the overly represented class to get a more balanced dataset. In general, and especially when the data is highly imbalanced, we recommend using more advanced methods.

## Split data into training and validation sets

We split the data into training data and validation data. We used `seed=1` to make sure everyone gets the same results. We will use the validation data to help us select model parameters.

In [10]:
train_data, validation_data = loans_data.random_split(.8, seed=1)

# Gradient boosted tree classifier

Gradient boosted trees are a powerful variant of boosting methods; they have been used to win many [Kaggle](https://www.kaggle.com/) competitions, and have been widely used in industry.  We will explore the predictive power of multiple decision trees as opposed to a single decision tree.

**Additional reading:** If you are interested in gradient boosted trees, here is some additional reading material:
* [GraphLab Create user guide](https://dato.com/learn/userguide/supervised-learning/boosted_trees_classifier.html)
* [Advanced material on boosted trees](http://homes.cs.washington.edu/~tqchen/pdf/BoostedTree.pdf)


We will now train models to predict `safe_loans` using the features above. In this section, we will experiment with training an ensemble of 5 trees. To cap the ensemble classifier at 5 trees, we call the function with **max_iterations=5** (recall that each iterations corresponds to adding a tree). We set `validation_set=None` to make sure everyone gets the same results.

In [11]:
model_5 = graphlab.boosted_trees_classifier.create(train_data, validation_set=None, 
        target = target, features = features, max_iterations = 5)

+-----------+--------------+-------------------+-------------------+

| 5         | 0.524094     | 0.673339          | 0.606229          |

| 4         | 0.419817     | 0.668476          | 0.613783          |

| 3         | 0.327348     | 0.664983          | 0.623206          |

| 2         | 0.227494     | 0.656976          | 0.636157          |

...

Number of feature columns   : 24

Number of classes           : 2

Number of examples          : 37219

--------------------------------------------------------

Boosted trees classifier:

# Making predictions

Just like we did in previous sections, let us consider a few positive and negative examples **from the validation set**. We will do the following:
* Predict whether or not a loan is likely to default.
* Predict the probability with which the loan is likely to default.

In [12]:
# Select all positive and negative examples.
validation_safe_loans = validation_data[validation_data[target] == 1]
validation_risky_loans = validation_data[validation_data[target] == -1]

# Select 2 examples from the validation set for positive & negative loans
sample_validation_data_risky = validation_risky_loans[0:2]
sample_validation_data_safe = validation_safe_loans[0:2]

# Append the 4 examples into a single dataset
sample_validation_data = sample_validation_data_safe.append(sample_validation_data_risky)
sample_validation_data

Columns:
	safe_loans	int
	grade	str
	sub_grade_num	float
	short_emp	int
	emp_length_num	int
	home_ownership	str
	dti	float
	purpose	str
	payment_inc_ratio	float
	delinq_2yrs	int
	delinq_2yrs_zero	int
	inq_last_6mths	int
	last_delinq_none	int
	last_major_derog_none	int
	open_acc	int
	pub_rec	int
	pub_rec_zero	int
	revol_util	float
	total_rec_late_fee	float
	int_rate	float
	total_rec_int	float
	annual_inc	int
	funded_amnt	int
	funded_amnt_inv	int
	installment	float

Rows: 4

Data:
+------------+-------+---------------+-----------+----------------+----------------+
| safe_loans | grade | sub_grade_num | short_emp | emp_length_num | home_ownership |
+------------+-------+---------------+-----------+----------------+----------------+
|     1      |   B   |      0.2      |     0     |       3        |    MORTGAGE    |
|     1      |   B   |      0.6      |     1     |       1        |      RENT      |
|     -1     |   D   |      0.4      |     0     |       3        |      RENT      |
|     

### Predicting on sample validation data

For each row in the **sample_validation_data**, write code to make **model_5** predict whether or not the loan is classified as a **safe loan**.

**Hint:** Use the `predict` method in `model_5` for this.

In [76]:
sample_validation_data[target]

dtype: int
Rows: 4
[1, 1, -1, -1]

In [86]:
model_5_predictions = numpy.array([model_5.predict(row)[0] for row in sample_validation_data])
print((model_5_predictions == sample_validation_data[target]))

[ True  True  True False]


In [90]:
print("correct: {0} %".format((model_5_predictions == sample_validation_data[target]).sum()/float(len(model_5_predictions)) * 100))

correct: 75.0 %


**Quiz question:** What percentage of the predictions on `sample_validation_data` did `model_5` get correct?

### Prediction probabilities

For each row in the **sample_validation_data**, what is the probability (according **model_5**) of a loan being classified as **safe**? 

**Hint:** Set `output_type='probability'` to make **probability** predictions using `model_5` on `sample_validation_data`:

In [94]:
model_5_probabilities = numpy.array([model_5.predict(row, output_type='probability')[0] for row in sample_validation_data])
print(model_5_probabilities)

[ 0.70459056  0.59634089  0.44925159  0.61190999]


**Quiz Question:** According to **model_5**, which loan is the least likely to be a safe loan?

In [103]:
lowest_probability = model_5_probabilities.min()
print("Index of least likely to be safe: {0}".format(numpy.where(lowest_probability == model_5_probabilities)[0][0]))

Index of least likely to be safe: 2


**Checkpoint:** Can you verify that for all the predictions with `probability >= 0.5`, the model predicted the label **+1**?

In [105]:
LIKELIHOOD_THRESHOLD = 0.5
for index, probability in enumerate(model_5_probabilities):
    if probability >= LIKELIHOOD_THRESHOLD:
        assert model_5_predictions[index] == 1
    else:
        assert model_5_predictions[index] == -1

## Evaluating the model on the validation data

Recall that the accuracy is defined as follows:
$$
\mbox{accuracy} = \frac{\mbox{# correctly classified examples}}{\mbox{# total examples}}
$$

Evaluate the accuracy of the **model_5** on the **validation_data**.

**Hint**: Use the `.evaluate()` method in the model.

In [109]:
model_5_evaluation = model_5.evaluate(validation_data)
print(model_5_evaluation['accuracy'])

0.668138733305


Calculate the number of **false positives** made by the model.

In [120]:
confusion_matrix = model_5_evaluation['confusion_matrix']
SAFE, RISKY = 1, -1
predicted_safe = confusion_matrix[confusion_matrix['predicted_label'] == SAFE]
FALSE_POSITIVES  = predicted_safe[predicted_safe['target_label'] == RISKY]['count'][0]
print("Number Of false positives: {0}".format(FALSE_POSITIVES))

Number Of false positives: 1618


**Quiz question**: What is the number of **false positives** on the **validation_data**?

Calculate the number of **false negatives** made by the model.

In [121]:
predicted_risky = confusion_matrix[confusion_matrix['predicted_label'] == RISKY]
FALSE_NEGATIVES = predicted_risky[predicted_risky['target_label'] ==SAFE]['count'][0]
print('Number of false negatives: {0}'.format(FALSE_NEGATIVES))

Number of false negatives: 1463


## Comparison with decision trees

In the earlier assignment, we saw that the prediction accuracy of the decision trees was around **0.64** (rounded). In this assignment, we saw that **model_5** has an accuracy of **0.67** (rounded).

Here, we quantify the benefit of the extra 3% increase in accuracy of **model_5** in comparison with a single decision tree from the original decision tree assignment.

As we explored in the earlier assignment, we calculated the cost of the mistakes made by the model. We again consider the same costs as follows:

* **False negatives**: Assume a cost of \$10,000 per false negative.
* **False positives**: Assume a cost of \$20,000 per false positive.

Assume that the number of false positives and false negatives for the learned decision tree was

* **False negatives**: 1936
* **False positives**: 1503

Using the costs defined above and the number of false positives and false negatives for the decision tree, we can calculate the total cost of the mistakes made by the decision tree model as follows:

```
cost = $10,000 * 1936  + $20,000 * 1503 = $49,420,000
```

The total cost of the mistakes of the model is $49.42M. That is a **lot of money**!.

**Quiz Question**: Using the same costs of the false positives and false negatives, what is the cost of the mistakes made by the boosted tree model (**model_5**) as evaluated on the **validation_set**?

In [129]:
FALSE_NEGATIVE_COST, FALSE_POSITIVE_COST = 10000, 20000
decision_tree_cost = FALSE_NEGATIVE_COST * 1936  + FALSE_POSITIVE_COST * 1503
model_5_cost = FALSE_POSITIVES * FALSE_POSITIVE_COST + FALSE_NEGATIVES * FALSE_NEGATIVE_COST
decision_tree_cost = 49420000
print('Decision Tree Cost: $ {0:,}'.format(decision_tree_cost))
print("Model 5 Cost: $ {0:,}".format(model_5_cost))
print("Cost Improvement: $ {0:,}".format(decision_tree_cost - model_5_cost))

Decision Tree Cost: $ 49,420,000
Model 5 Cost: $ 46,990,000
Cost Improvement: $ 2,430,000


**Reminder**: Compare the cost of the mistakes made by the boosted trees model with the decision tree model. The extra 3% improvement in prediction accuracy can translate to several million dollars!  And, it was so easy to get by simply boosting our decision trees.

## Most positive & negative loans.

In this section, we will find the loans that are most likely to be predicted **safe**. We can do this in a few steps:

* **Step 1**: Use the **model_5** (the model with 5 trees) and make **probability predictions** for all the loans in the **validation_data**.
* **Step 2**: Similar to what we did in the very first assignment, add the probability predictions as a column called **predictions** into the validation_data.
* **Step 3**: Sort the data (in descreasing order) by the probability predictions.

Start here with **Step 1** & **Step 2**. Make predictions using **model_5** for examples in the **validation_data**. Use `output_type = probability`.

In [131]:
validation_data['predictions'] = model_5.predict(validation_data, output_type='probability')

**Checkpoint:** For each row, the probabilities should be a number in the range **[0, 1]**. We have provided a simple check here to make sure your answers are correct.

In [134]:
print "Your loans      : %s\n" % validation_data['predictions'].head(4)
print "Expected answer : %s" % [0.4492515948736132, 0.6119100103640573,
                                0.3835981314851436, 0.3693306705994325]
assert validation_data['predictions'].head(4) == [0.4492515948736132,
                                                  0.6119100103640573,
                                                  0.3835981314851436,
                                                  0.3693306705994325]

Your loans      : [0.44925159215927124, 0.6119099855422974, 0.38359811902046204, 0.3693307042121887]

Expected answer : [0.4492515948736132, 0.6119100103640573, 0.3835981314851436, 0.3693306705994325]


Now, we are ready to go to **Step 3**. You can now use the `prediction` column to sort the loans in **validation_data** (in descending order) by prediction probability. Find the top 5 loans with the highest probability of being predicted as a **safe loan**.

In [146]:
validation_data = validation_data.sort('predictions', ascending=False)

In [147]:
print(validation_data[:5])

+------------+-------+---------------+-----------+----------------+----------------+
| safe_loans | grade | sub_grade_num | short_emp | emp_length_num | home_ownership |
+------------+-------+---------------+-----------+----------------+----------------+
|     1      |   A   |      0.2      |     0     |       11       |    MORTGAGE    |
|     1      |   A   |      0.4      |     0     |       4        |    MORTGAGE    |
|     1      |   A   |      0.6      |     0     |       11       |    MORTGAGE    |
|     1      |   A   |      0.6      |     1     |       1        |    MORTGAGE    |
|     1      |   A   |      0.4      |     0     |       4        |    MORTGAGE    |
+------------+-------+---------------+-----------+----------------+----------------+
+-------+------------------+-------------------+-------------+------------------+
|  dti  |     purpose      | payment_inc_ratio | delinq_2yrs | delinq_2yrs_zero |
+-------+------------------+-------------------+-------------+---------

** Quiz question**: What grades are the top 5 loans?

Grade A

Let us repeat this excercise to find the top 5 loans (in the **validation_data**) with the **lowest probability** of being predicted as a **safe loan**:

In [153]:
ascending = validation_data.sort('predictions')

In [152]:
ascending[:5]['grade']

dtype: str
Rows: 5
['C', 'C', 'D', 'C', 'B']

**Checkpoint:** You should expect to see 5 loans with the grade ['**D**', '**C**', '**C**', '**C**', '**B**'].

## Effect of adding more trees

In this assignment, we will train 5 different ensemble classifiers in the form of gradient boosted trees. We will train models with 10, 50, 100, 200, and 500 trees.  We use the **max_iterations** parameter in the boosted tree module. 

Let's get sarted with a model with **max_iterations = 10**:

In [154]:
model_10 = graphlab.boosted_trees_classifier.create(train_data, validation_set=None, 
                                                    target = target, features = features, max_iterations = 10, verbose=False)

Now, train 4 models with **max_iterations** to be:
* `max_iterations = 50`, 
* `max_iterations = 100`
* `max_iterations = 200`
* `max_iterations = 500`. 

Let us call these models **model_50**, **model_100**, **model_200**, and **model_500**. You can pass in `verbose=False` in order to suppress the printed output.

**Warning:** This could take a couple of minutes to run.

In [163]:
MAX_ITERATIONS = (50, 100, 200, 500)
models = {"model_{0}".format(max_iterations): graphlab.boosted_trees_classifier.create(train_data,
                                                                                       validation_set=None,
                                                                                       target=target,
                                                                                       features=features,
                                                                                       max_iterations=max_iterations,
                                                                                       verbose=False) for max_iterations in MAX_ITERATIONS}
print(models)

{'model_50': Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 37219
Number of feature columns     : 24
Number of unpacked features   : 24
Number of classes             : 2

Settings
--------
Number of trees               : 50
Max tree depth                : 6
Training time (sec)           : 5.1084
Training accuracy             : 0.7539
Validation accuracy           : None
Training log_loss             : 0.5078
Validation log_loss           : None
, 'model_500': Class                         : BoostedTreesClassifier

Schema
------
Number of examples            : 37219
Number of feature columns     : 24
Number of unpacked features   : 24
Number of classes             : 2

Settings
--------
Number of trees               : 500
Max tree depth                : 6
Training time (sec)           : 52.2898
Training accuracy             : 0.9618
Validation accuracy           : None
Training log_loss             : 0.217
Validation log_loss       

## Compare accuracy on entire validation set

Now we will compare the predicitve accuracy of our models on the validation set. Evaluate the **accuracy** of the 10, 50, 100, 200, and 500 tree models on the **validation_data**. Use the `.evaluate` method.

In [190]:
models['model_10'] = model_10
MAX_ITERATIONS = (10,) + MAX_ITERATIONS

In [192]:
evaluations = {}

for max_iterations in MAX_ITERATIONS:
    model_name = 'model_{0}'.format(max_iterations)
    print(model_name)
    print('-' * len(model_name))
    evaluations[model_name] = models[model_name].evaluate(validation_data)
    print(evaluations[model_name]['accuracy'])
    print('')

0.671800947867



0.684510986644

model_500
---------


0.691727703576

model_200
---------


0.690758293839

model_100
---------


model_10
--------
0.672770357604

model_50
--------


In [196]:
accuracies = pandas.DataFrame.from_dict([{model_name: evaluation['accuracy'] for model_name, evaluation in evaluations.iteritems()}])
axe = accuracies.plot(kind='bar')

**Quiz Question:** Which model has the **best** accuracy on the **validation_data**?

In [197]:
best_accuracy, best_model = max((value['accuracy'], key) for key, value in evaluations.iteritems())
print("Best Model: {0}\tAccuracy: {1:.2f} %".format(best_model, best_accuracy * 100))

Best Model: model_100	Accuracy: 69.17 %


**Quiz Question:** Is it always true that the model with the most trees will perform best on test data?

No

## Plot the training and validation error vs. number of trees

Recall from the lecture that the classification error is defined as

$$
\mbox{classification error} = 1 - \mbox{accuracy} 
$$

In this section, we will plot the **training and validation errors versus the number of trees** to get a sense of how these models are performing. We will compare the 10, 50, 100, 200, and 500 tree models. You will need [matplotlib](http://matplotlib.org/downloads.html) in order to visualize the plots. 

First, make sure this block of code runs on your computer.

In [185]:
import matplotlib.pyplot as plt
%matplotlib inline
def make_figure(dim, title, xlabel, ylabel, legend):
    plt.rcParams['figure.figsize'] = dim
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    if legend is not None:
        plt.legend(loc=legend, prop={'size':15})
    plt.rcParams.update({'font.size': 16})
    plt.tight_layout()

In order to plot the classification errors (on the **train_data** and **validation_data**) versus the number of trees, we will need lists of these accuracies, which we get by applying the method `.evaluate`. 

**Steps to follow:**

* **Step 1:** Calculate the classification error for model on the training data (**train_data**).
* **Step 2:** Store the training errors into a list (called `training_errors`) that looks like this:
```
[train_err_10, train_err_50, ..., train_err_500]
```
* **Step 3:** Calculate the classification error of each model on the validation data (**validation_data**).
* **Step 4:** Store the validation classification error into a list (called `validation_errors`) that looks like this:
```
[validation_err_10, validation_err_50, ..., validation_err_500]
```
Once that has been completed, the rest of the code should be able to evaluate correctly and generate the plot.


Let us start with **Step 1**. Write code to compute the classification error on the **train_data** for models **model_10**, **model_50**, **model_100**, **model_200**, and **model_500**.

In [198]:
training_evaluations = {}
for max_iterations in MAX_ITERATIONS:
    model_name = "model_{0}".format(max_iterations)
    print(model_name)
    print('=' * len(model_name))
    training_evaluations[model_name] = models[model_name].evaluate(train_data)    
    accuracy = training_evaluations[model_name]['accuracy']
    print("Classification Error: {0}".format(1 - accuracy))
    print('')

Classification Error: 0.0382331604825



Classification Error: 0.136435691448

model_500


Classification Error: 0.20043526156

model_200


Classification Error: 0.246057121363

model_100


Classification Error: 0.311749375319

model_50


model_10


Now, let us run **Step 2**. Save the training errors into a list called **training_errors**

In [200]:
training_errors = [1 - training_evaluations['model_{0}'.format(max_iterations)]['accuracy'] for max_iterations in MAX_ITERATIONS]
print(training_errors)

[0.31174937531905744, 0.24605712136274482, 0.20043526155995595, 0.13643569144791634, 0.03823316048254921]


Now, onto **Step 3**. Write code to compute the classification error on the **validation_data** for models **model_10**, **model_50**, **model_100**, **model_200**, and **model_500**.

In [201]:
validation_errors = [1 - evaluations['model_{0}'.format(max_iterations)]['accuracy'] for max_iterations in MAX_ITERATIONS]
print(validation_errors)

[0.3272296423955192, 0.30924170616113744, 0.30827229642395515, 0.31548901335631196, 0.3281990521327014]


Now, let us run **Step 4**. Save the training errors into a list called **validation_errors**

Now, we will plot the **training_errors** and **validation_errors** versus the number of trees. We will compare the 10, 50, 100, 200, and 500 tree models. We provide some plotting code to visualize the plots within this notebook. 

Run the following code to visualize the plots.

In [202]:
plt.plot([10, 50, 100, 200, 500], training_errors, linewidth=4.0, label='Training error')
plt.plot([10, 50, 100, 200, 500], validation_errors, linewidth=4.0, label='Validation error')

make_figure(dim=(10,5), title='Error vs number of trees',
            xlabel='Number of trees',
            ylabel='Classification error',
            legend='best')

**Quiz question**: Does the training error reduce as the number of trees increases?

Yes

**Quiz question**: Is it always true that the validation error will reduce as the number of trees increases?

No